# Using Placeholders to Prevent SQL Injection

In this notebook, you'll learn about SQL injection vulnerabilities and how to prevent them using placeholders (parameterized queries) in SQLite with Python.

In [ ]:
import sqlite3

# Connect to database
conn = sqlite3.connect('security_demo.db')
cursor = conn.cursor()

# Create a users table for demonstration
cursor.execute('''
    CREATE TABLE users (
        id INTEGER PRIMARY KEY,
        username TEXT UNIQUE,
        password TEXT,
        email TEXT
    )
''')

# Insert some test data
users_data = [
    ('alice', 'password123', 'alice@example.com'),
    ('bob', 'secret456', 'bob@example.com'),
    ('admin', 'adminpass', 'admin@example.com')
]

cursor.executemany('INSERT INTO users (username, password, email) VALUES (?, ?, ?)', users_data)

print('Database and test data created')

## What is SQL Injection?

SQL injection is a code injection technique that exploits vulnerabilities in an application's software by inserting malicious SQL code into a query. This can lead to:

- Unauthorized data access
- Data modification
- Data deletion
- Database server compromise

SQL injection occurs when user input is directly concatenated into SQL statements.

## Vulnerable Code Example

Here's an example of vulnerable code that directly concatenates user input:

In [ ]:
def vulnerable_login(username, password):
    # DANGEROUS: Direct string concatenation
    query = f"SELECT * FROM users WHERE username = '{username}' AND password = '{password}'"
    cursor.execute(query)
    return cursor.fetchall()

print('Vulnerable login function defined')

# Normal usage
result = vulnerable_login('alice', 'password123')
print('Normal login result:', len(result), 'users found')

# SQL injection attack
malicious_username = "' OR '1'='1"  # This will match all users
malicious_password = "' OR '1'='1"

result = vulnerable_login(malicious_username, malicious_password)
print('SQL injection result:', len(result), 'users found (should be 0!)')
print('Compromised data:', result)

## Using Placeholders (Parameterized Queries)

The solution is to use placeholders in SQL queries and pass parameters separately. SQLite supports two placeholder styles:

- `?` - Positional placeholders
- `:name` - Named placeholders

## Positional Placeholders

Use `?` for positional parameters:

In [ ]:
def secure_login_positional(username, password):
    # SECURE: Using positional placeholders
    query = "SELECT * FROM users WHERE username = ? AND password = ?"
    cursor.execute(query, (username, password))
    return cursor.fetchall()

print('Secure login function (positional) defined')

# Normal usage
result = secure_login_positional('alice', 'password123')
print('Normal login result:', len(result), 'users found')

# Attempted SQL injection (now safe)
result = secure_login_positional(malicious_username, malicious_password)
print('SQL injection attempt result:', len(result), 'users found (correctly 0)')
print('Data protected:', result)

## Named Placeholders

Use `:name` for named parameters:

In [ ]:
def secure_login_named(username, password):
    # SECURE: Using named placeholders
    query = "SELECT * FROM users WHERE username = :user AND password = :pass"
    cursor.execute(query, {'user': username, 'pass': password})
    return cursor.fetchall()

print('Secure login function (named) defined')

# Test both normal and malicious input
result = secure_login_named('alice', 'password123')
print('Normal login result:', len(result), 'users found')

result = secure_login_named(malicious_username, malicious_password)
print('SQL injection attempt result:', len(result), 'users found (correctly 0)')

## Other SQL Operations with Placeholders

Placeholders work with all SQL operations: INSERT, UPDATE, DELETE, SELECT

In [ ]:
# INSERT with placeholders
new_user = ('charlie', 'charliepass', 'charlie@example.com')
cursor.execute('INSERT INTO users (username, password, email) VALUES (?, ?, ?)', new_user)
print('User inserted securely')

# UPDATE with placeholders
cursor.execute('UPDATE users SET email = ? WHERE username = ?', ('newemail@example.com', 'bob'))
print('User updated securely')

# DELETE with placeholders
cursor.execute('DELETE FROM users WHERE username = ?', ('charlie',))
print('User deleted securely')

# SELECT with LIKE and placeholders
search_term = 'a%'
cursor.execute('SELECT username FROM users WHERE username LIKE ?', (search_term,))
results = cursor.fetchall()
print('Users starting with "a":', [row[0] for row in results])

## executemany() with Placeholders

Use placeholders with executemany() for bulk operations:

In [ ]:
# Bulk insert with placeholders
new_users = [
    ('diana', 'dianapass', 'diana@example.com'),
    ('eve', 'evepass', 'eve@example.com')
]

cursor.executemany('INSERT INTO users (username, password, email) VALUES (?, ?, ?)', new_users)
print('Bulk insert completed securely')

# Bulk update with placeholders
updates = [
    ('alice_new@example.com', 'alice'),
    ('bob_new@example.com', 'bob')
]

cursor.executemany('UPDATE users SET email = ? WHERE username = ?', updates)
print('Bulk update completed securely')

## Dynamic Queries with Placeholders

Building dynamic queries while maintaining security:

In [ ]:
def search_users(username=None, email=None, limit=10):
    # Build query dynamically but safely
    conditions = []
    params = []
    
    if username:
        conditions.append('username LIKE ?')
        params.append(f'%{username}%')
    
    if email:
        conditions.append('email LIKE ?')
        params.append(f'%{email}%')
    
    where_clause = ' AND '.join(conditions) if conditions else '1=1'
    query = f'SELECT username, email FROM users WHERE {where_clause} LIMIT ?'
    params.append(limit)
    
    cursor.execute(query, params)
    return cursor.fetchall()

print('Dynamic search function defined')

# Test the function
results = search_users(username='a')
print('Users with "a" in username:', results)

results = search_users(email='example.com')
print('Users with "example.com" in email:', results)

## Common SQL Injection Attack Vectors

Understanding common attack patterns:

In [ ]:
# Example attack vectors (don't use these!)
attack_vectors = [
    "' OR '1'='1",           # Bypass authentication
    "'; DROP TABLE users;--", # Delete table
    "' UNION SELECT * FROM secret_table;--", # Union-based injection
    "admin'--",               # Comment out rest of query
    "1; UPDATE users SET password='hacked'--" # Multiple statements
]

print('Common SQL injection attack vectors:')
for i, vector in enumerate(attack_vectors, 1):
    print(f'{i}. {vector}')

# Test that placeholders prevent these attacks
def test_attack(vector):
    try:
        cursor.execute('SELECT * FROM users WHERE username = ?', (vector,))
        result = cursor.fetchall()
        return len(result)
    except:
        return 'ERROR'

print('\nTesting attack vectors with placeholders:')
for vector in attack_vectors:
    safe_result = test_attack(vector)
    print(f'Attack "{vector[:20]}...": {safe_result} results (safe!)')

## Best Practices

- **Always use placeholders** for user input in SQL queries
- **Never concatenate user input** directly into SQL strings
- **Validate input** on the application side as well
- **Use parameterized queries** for all dynamic SQL
- **Limit database permissions** to minimize damage from successful injections
- **Log and monitor** database queries for suspicious activity
- **Keep database drivers updated** for security patches

In [ ]:
# Final verification - show all users
cursor.execute('SELECT username, email FROM users ORDER BY username')
all_users = cursor.fetchall()

print('Final user list:')
for user in all_users:
    print(f'  {user[0]}: {user[1]}')

# Commit changes
conn.commit()

## Summary

In this notebook, you learned:

- What SQL injection is and why it's dangerous
- How vulnerable string concatenation leads to security breaches
- How to use positional (`?`) and named (`:name`) placeholders
- How placeholders work with INSERT, UPDATE, DELETE, and SELECT
- How to use placeholders with executemany() for bulk operations
- How to build dynamic queries safely
- Common SQL injection attack vectors and how placeholders prevent them
- Best practices for secure database programming

**Key takeaway**: Always use parameterized queries with placeholders. Never concatenate user input directly into SQL strings. This simple practice prevents the most common and dangerous security vulnerability in database applications.

In [ ]:
# Clean up
conn.close()
print('Database connection closed')